# Mais detalhes sobre a biblioteca JuMP

O objetivo deste tutorial é demonstrar alguns recursos extras da biblioteca JuMP:
- A descrição (e solução) de problemas não-lineares
- O uso de valores iniciais como "dica" para o solver
- A construção de um modelo sem solver

## Pacotes necessários

Este tutorial requer os seguintes pacotes:

In [1]:
using JuMP
import HiGHS
import Ipopt

## Valores iniciais para as variáveis

Vamos continuar o problema da mochila:
$$
\begin{aligned}
\max \; & c^\top x       \\
\text{s.t.} \; & w^\top x \le C \\
        & x \text{ binária }.
\end{aligned}
$$

Agora, iremos usar uma formulação alternativa para a restrição binária: Vamos usar que $x^2 = x \Leftrightarrow x \in \{0,1\}$.
Também incluiremos a restrição $0 \leq x \leq 1$.

### Dados

In [2]:
n = 5;
capacidade = 10.0;
lucro = [5.0, 3.0, 2.0, 7.0, 4.0];
peso = [2.0, 8.0, 4.0, 2.0, 5.0];

### Formulação JuMP

Podemos, diretamente na construção do modelo, dar um valor inicial para as variáveis, usando a sintaxe `start = VALOR`.

In [3]:
modelo = Model(Ipopt.Optimizer)
@variable(modelo, 0 <= x[1:n] <= 1, start=1.0)
@constraint(modelo, sum(peso[i] * x[i] for i in 1:n) <= capacidade)
@constraint(modelo, x.^2 .== x)
@objective(modelo, Max, sum(lucro[i] * x[i] for i in 1:n));

No caso de uma variável vetorial, como feito acima, demos o mesmo valor inicial para todas as coordenadas.

Também seria possível dar valores iniciais diferentes, por exemplo usando

```julia
x_start = [1, 0, 1, 0, 1]
@variable(modelo, 0 <= x[i=1:n] <= 1, start=x_start[i])
```

Note que também indicamos um índice (o `i`) para a variável, o que é necessário para que o JuMP possa gerar valores diferentes para cada coordenada na expressão depois de `start`.

In [4]:
print(modelo)

Vamos resolver com este ponto inicial:

In [5]:
optimize!(modelo)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       10
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        5
                     variables with only upper bounds:        0
Total number of equality constraints.................:        5
Total number of inequality c

E ver algumas informações mais gerais sobre a solução:

In [6]:
solution_summary(modelo)

* Solver : Ipopt

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "Solve_Succeeded"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.40000e+01
  Dual objective value : 4.91517e-14

* Work counters
  Solve time (sec)   : 3.09999e-02
  Barrier iterations : 23


Os itens escolhidos são

In [7]:
itens_escolhidos = [i for i in 1:n if value(x[i]) > 0.5]

3-element Vector{Int64}:
 1
 3
 4

Também é possível modificar o valor inicial das variáveis de decisão, o que pode mudar a solução do problema:

In [8]:
set_start_value.(x, 0.5)

5-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing

In [9]:
optimize!(modelo)

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       10
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        5
                     variables with only upper bounds:        0
Total number of equality constraints.................:        5
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0500000e+01 5.00e-01 7.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [10]:
itens_escolhidos = [i for i in 1:n if value(x[i]) > 0.5]

3-element Vector{Int64}:
 1
 4
 5

## Exercícios

* Teste outros valores iniciais, como por exemplo $0$, ou um vetor de condições iniciais como [0, 1, 1, 0, 0]. O que você observa?

## A função de Rosenbrock

O problema de Rosenbrock é um problema clássico de otimização não-linear:
$$
\min\limits_{x,y} \quad (1-x)^2 + 100(y-x^2)^2
$$


In [11]:
m = JuMP.Model()
@variable(m, x)
@variable(m, y)
@objective(m, Min, (x - 1)^2 + 100*(y - x^2)^2)
print(m)

In [12]:
# A função objetivo é não-linear, e, mais, não quadrática, e o HiGHS não suporta isso
try
    set_optimizer(m, HiGHS.Optimizer)
    optimize!(m)
catch err
    showerror(stderr, err)
end


Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms


MathOptInterface.UnsupportedAttribute{MathOptInterface.ObjectiveFunction{MathOptInterface.ScalarNonlinearFunction}}: Attribute MathOptInterface.ObjectiveFunction{MathOptInterface.ScalarNonlinearFunction}() is not supported by the model.

In [13]:
set_optimizer(m, Ipopt.Optimizer)
optimize!(m)

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0000000e+00 0.00e+00 2.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [14]:
value.([x, y])

2-element Vector{Float64}:
 0.9999999999956423
 0.9999999999878554

## Exercícios

* A função de Rosenbrock pode ser difícil de otimizar numericamente.
  Tente resolver o problema com diferentes valores iniciais, e veja se algum converge para diferentes mínimos locais.